
#Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

### We will be provided a company name and their primary website.

### Step 0: Setup Ollama Server (Background)


In [1]:
# 1. Install Ollama in Colab
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start Ollama server in the background
import subprocess
import time

# Start server and suppress output to keep the notebook clean
subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# 3. Wait for the server to initialize
time.sleep(10)
print("Ollama server is running in the background!")

>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tgz
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Ollama server is running in the background!


### Cell 1: Imports and Scraper Functions


In [2]:
# Install necessary libraries for scraping and AI calls
!pip install openai beautifulsoup4 requests

import os
import json
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
from IPython.display import Markdown, display, update_display

# --- Define Scraper Functions (Replacement for Ed's scraper.py) ---

def fetch_website_contents(url):
    """
    Fetches the visible text content from a given URL by removing scripts and styles.
    """
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Remove non-content tags
        for script_or_style in soup(["script", "style"]):
            script_or_style.extract()

        # Extract and clean text
        return soup.get_text(separator="\n", strip=True)
    except Exception as e:
        return f"Error fetching content: {e}"

def fetch_website_links(url):
    """
    Finds all relevant hyperlinks on a page and converts relative URLs to absolute ones.
    """
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all 'a' tags with href attributes
        links = [a.get('href') for a in soup.find_all('a', href=True)]

        # Standardize URLs (handle relative links like /about)
        base_url = url.rstrip('/')
        full_links = []
        for link in links:
            if link.startswith('http'):
                full_links.append(link)
            elif link.startswith('/'):
                full_links.append(f"{base_url}{link}")

        # Return unique links only
        return list(set(full_links))
    except Exception as e:
        print(f"Error fetching links: {e}")
        return []

print("Cell 1: Scraper functions and libraries are ready!")

Cell 1: Scraper functions and libraries are ready!


### Cell 2: Initialize Model and API Client

In [3]:
# 1. Download the specific model for our project
# We are using llama3.2:1b because it's fast and lightweight for testing
!ollama pull llama3.2:1b

# 2. Initialize the OpenAI client to connect to the local Ollama server
# Ollama provides an OpenAI-compatible endpoint at localhost:11434
client = OpenAI(base_url="http://localhost:11434/v1", api_key='ollama')

# 3. Define the constant for our model name
# Ed used 'gpt-5-nano' but we are using our free local model
MODEL = "llama3.2:1b"

print(f"Cell 2: Client initialized and model {MODEL} is ready!")


Cell 2: Client initialized and model llama3.2:1b is ready!


### Cell 3: Testing the Scraper (Raw Links Fetch)

In [4]:
# Define the target URL & Company Name for our brochure
target_url = "https://huggingface.co/"
company_name = "Hugging Face"

# Call our scraper function to get all raw links from the page
# This is a standard Python step, no AI is involved yet
raw_links = fetch_website_links(target_url)

# Display the total count and the first few links to verify the scraper is working
print(f"Total links found on {target_url}: {len(raw_links)}")
print("First 15 links found:")
for link in raw_links[:15]:
    print(f"- {link}")

Total links found on https://huggingface.co/: 63
First 15 links found:
- https://endpoints.huggingface.co
- https://huggingface.co/Intel
- https://huggingface.co/brand
- https://apply.workable.com/huggingface/
- https://huggingface.co/join
- https://huggingface.co/huggingface
- https://huggingface.co/Lightricks/LTX-2
- https://huggingface.co/grammarly
- https://huggingface.co/docs/smolagents
- https://huggingface.co/docs/huggingface_hub
- https://huggingface.co/docs/transformers
- https://huggingface.co/docs/accelerate
- https://huggingface.co/spaces/Wan-AI/Wan2.2-Animate
- https://huggingface.co/docs/trl
- https://twitter.com/huggingface


### Cell 4: Defining the Link Selection System Prompt

In [5]:
# Define the system prompt for the link selection step
# This uses 'One-shot prompting' by providing a single example of the desired JSON output
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

### Cell 5: The User Prompt Function

In [6]:
def get_links_user_prompt(url):
    """
    Combines instructions with raw scraped links to create a final user prompt.
    """
    # Start the prompt with specific instructions for the LLM
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, or email links.

Links (some might be relative links):

"""
    # Use our scraper to get the actual list of links from the website
    links = fetch_website_links(url)

    # Append the list of links to the prompt, each on a new line
    user_prompt += "\n".join(links)

    return user_prompt

# Test the function to see what the final prompt looks like
# We are still not calling the AI yet; just building the 'Question'
test_prompt = get_links_user_prompt("https://huggingface.co/")
print(test_prompt)


Here is the list of links on the website https://huggingface.co/ -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, or email links.

Links (some might be relative links):

https://endpoints.huggingface.co
https://huggingface.co/Intel
https://huggingface.co/brand
https://apply.workable.com/huggingface/
https://huggingface.co/join
https://huggingface.co/huggingface
https://huggingface.co/Lightricks/LTX-2
https://huggingface.co/grammarly
https://huggingface.co/docs/smolagents
https://huggingface.co/docs/huggingface_hub
https://huggingface.co/docs/transformers
https://huggingface.co/docs/accelerate
https://huggingface.co/spaces/Wan-AI/Wan2.2-Animate
https://huggingface.co/docs/trl
https://twitter.com/huggingface
https://huggingface.co/join/discord
https://huggingface.co/pricing#spaces
https://huggingface.co/datasets/nvidia/PhysicalAI-Autonomous-Vehicles
https://h

### Cell 6: The First AI Call - Selecting Relevant Links

In [7]:
def select_relevant_links(url):
    """
    Calls the LLM to analyze links and return only the relevant ones in JSON format.
    """
    print(f"Selecting relevant links for {url} by calling {MODEL}...")

    # Making the API call to Ollama
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt}, # Instructions from Cell 4
            {"role": "user", "content": get_links_user_prompt(url)} # Prompt with links from Cell 5
        ],
        # Forcing the model to respond with a valid JSON object
        response_format={"type": "json_object"}
    )

    # Extract the raw text result from the response
    result = response.choices[0].message.content

    # Convert the JSON string into a Python dictionary
    links_dict = json.loads(result)

    print(f"Found {len(links_dict['links'])} relevant links.")
    return links_dict

# Test the full function for your target URL
selected_links = select_relevant_links(target_url)
display(selected_links)

Selecting relevant links for https://huggingface.co/ by calling llama3.2:1b...
Found 2 relevant links.


{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'company', 'url': 'https://full.url goes here'}]}

### Cell 7: Researching - Fetching Content from All Relevant Links

In [8]:
def fetch_page_and_all_relevant_links(url):
    """
    Scrapes the main page and all relevant sub-pages identified by the AI in Cell 6.
    It combines everything into one large text block for the final brochure.
    """
    # 1. First, fetch the main landing page content using our scraper
    landing_page_content = fetch_website_contents(url)

    # 2. Get the list of relevant links using our AI function from Cell 6
    # This is an agentic pattern: using one AI call's output to drive the next steps
    relevant_links_json = select_relevant_links(url)

    # 3. Initialize a string to hold all the gathered information
    aggregated_content = f"## Landing Page Content:\n\n{landing_page_content}\n\n## Relevant Sub-Pages Information:\n"

    # 4. Loop through each relevant link and fetch its content
    for link in relevant_links_json['links']:
        print(f"Reading: {link['type']} at {link['url']}...")
        page_text = fetch_website_contents(link['url'])

        # Stitch the new page content into our big text block
        aggregated_content += f"\n\n### Sub-Page: {link['type']}\n{page_text}"

    return aggregated_content

# Execute the research phase
# Note: This might take 30-60 seconds as it visits multiple pages
all_data_for_brochure = fetch_page_and_all_relevant_links(target_url)

print("\n--- Research Complete! ---")
print(f"Total characters gathered for context: {len(all_data_for_brochure)}")

Selecting relevant links for https://huggingface.co/ by calling llama3.2:1b...
Found 2 relevant links.
Reading: about page at https://full.url/goes/here/about...
Reading: models at https://huggingface.co/models...

--- Research Complete! ---
Total characters gathered for context: 7923


### Cell 8: Defining Brochure Prompts and the English Generator

In [9]:
# 1. Define the system prompt for the brochure creation
# This tells the LLM the tone and structure we want
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# 2. Function to generate the user prompt for the brochure
def get_brochure_user_prompt(company_name, url, context_data):
    """
    Combines the company name with the researched data from Cell 7.
    """
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here is the researched content from their website {url}:\n\n"
    user_prompt += context_data
    # Truncate to 5,000 characters to manage context window and costs
    return user_prompt[:5000]

# 3. Function to create the English Brochure
def create_english_brochure(company_name, url, context_data):
    """
    Makes the first LLM call to create the brochure in English.
    """
    print(f"Generating English brochure for {company_name}...")
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, context_data)}
        ]
    )
    return response.choices[0].message.content

# Generate and display the English brochure
english_brochure = create_english_brochure(company_name, target_url, all_data_for_brochure)
display(Markdown("### English Brochure\n" + english_brochure))

Generating English brochure for Hugging Face...


### English Brochure
Hugging Face: The AI Community Building the Future
================================================================================

Welcome to Hugging Face, a collaborative platform for building and sharing artificial intelligence (AI) models and applications. Our community relies on your contributions to advance machine learning and improve lives.

**Mission**
----------

Our mission is to create an open source-based platform that enables the creation of high-quality AI models, datasets, and applications. We strive to foster collaboration among researchers, developers, and customers from diverse backgrounds.

**Our Values**
--------------

* **Community-driven**: We place the needs of our community at the forefront of everything we do.
* **Open-source**: Our mission depends on the success of this open source platform.
* **Accessible**: Anyone can contribute to our efforts and participate in making AI more accessible to all.
* **Innovative**: We challenge conventional wisdom and encourage new approaches in AI development.

**Join the Community**
--------------------

Whether you're a researcher, developer, customer, or enthusiast, we invite you to join us on this journey. With over 2 million models hosted across various platforms, our community is growing rapidly. You can:

* **Explore Models**: Browse through our collection of models, suitable for any AI task.
* **Discover Datasets**: Access and share datasets from leading providers.
* **Collaborate with Others**: Join spaces to discuss models, datasets, and applications.

**Accelerate Your ML**
----------------------

Our enterprise-grade platform provides:

* **Enterprise Solutions**: Dedicated support, access controls, and secure deployment options for large-scale AI operations.
* **Affordable Pricing**: Starting at $20/user/month, our solutions cater to various organization sizes.
* **Infinite Resources**: Utilize more than 50,000 organizations' contributions.

**Getting Started**
-------------------

Start developing with our library of Python clients. Join our Tutorials and Documentation sections for in-depth information on models, datasets, and platforms.

Our team also engages in:

*   **Research & Development**: State-of-the-art AI research and development to push the boundaries of machine learning.
*   **Open Meetings**: Collaborative meetings to discuss platform goals and progress.
*   **Community Engagement**: Organized hackathons and events to foster creativity and innovation.

**Meet Our Team**
-----------------

Our dedicated team consists of researchers, engineers, and community members working together to advance AI development. Learn about our leadership posts or follow recent updates from our social media channels:

```
Team
non-profit:
•   841 models
•   5.05k followers
AI at Meta (Enterprise):
•   2.29k models
•   10.6k followers
Amazon (Enterprise):
•   21 models
•   3.71k followers
Google (Enterprise):
•   1.07k models
•   41.6k followers
Intel (Enterprise):
•   269 models
•   3.34k followers
Microsoft (Enterprise):
•   431 models
•   17.7k followers
Grammarly (Team):
•   11 models
•   202 followers
Writer (Enterprise):
•   32 models
•   373 followers
Hugging Face Foundation:
```

**Learn More**
-------------

Discover the latest articles and research from our publication, the Hugging Face blog. We share insights into AI applications, model development, and platform strategies.

Contact us directly to stay informed about the most recent updates, discuss your projects, or collaborate on new ideas:

```
Press
Resources
```

Stay connected with Hugging Face in various formats and explore how you can be part of the AI innovation journey!

### Cell 9: The Third LLM Call - Urdu Translation

In [11]:
# 1. Define the system prompt for translation
# This instructs the LLM to provide a professional Urdu translation
translation_system_prompt = """
You are a professional translator.
Translate the provided English company brochure into clear, professional, and natural-sounding Urdu.
Maintain the markdown formatting (headings, bullets).
Ensure the tone is suitable for investors and customers.
"""

# 2. Function to translate the brochure using a third LLM call
def translate_to_urdu(english_text):
    """
    Takes the English brochure and sends it back to the LLM for Urdu translation.
    This is the 3rd step in our agentic chain.
    """
    print("Translating brochure to Urdu...")
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translation_system_prompt},
            {"role": "user", "content": f"Please translate this brochure into Urdu:\n\n{english_text}"}
        ]
    )
    return response.choices[0].message.content

# 3. Execute the translation and display the final result
urdu_brochure = translate_to_urdu(english_brochure)
display(Markdown("### Urdu Brochure\n" + urdu_brochure))

Translating brochure to Urdu...


### Urdu Brochure
# Hugging Face: The AI Community Building the Future

**ڈیجیٹل اسٹیلہٹ**

وہاں ہے Hugging Face، ایک cộngunsٹریٹڼ ڈویلپمنٹ پلیٹฟอรم جو ایسی آرن کا استعمال کر رہی ہے جس میں Artificial Intelligence (AI) ملازمتوں اور سروس کے منصوبوں کو فروغ دینے کو شامل کیا گیا ہے۔ یہ کمپیوٹERRسٹ کے لیپائڈ کی طرف سے وائرال ٹرنرنگ کی ایک اور نمائندہ پلیٹ فارم کی طرح کام کررہی ہے۔

```markdown
**میشن**
-----------
```

### مصنوعات کا تجزیہ

اسپیس مینجمنٹ کے لیپ اور سروس کے منصوبوں کے لیپ:
*   **اسپیس ویب Sarafa**: ایک مینجمنٹ کی ویب پلیٹ فارم جس پر وائرال ٹرنر بنانے کی صلاحیت تھی.
*   **اسپیس میڈیا Sarafa**: ایک منسلک پلیٹ فارم جو اسپیس ویب Sarafa سے ملازمت کا پروسیس کو ایک اور نمائندہ سروس سمجھ کر بناتا ہے۔

### کہانی

وہاں بھی اس کا تعلق ہے کہ مینجمنٹ نے ٹیویٹلی اور ایمٹی الیکٹرانکس جوئی کی موarde پر ایک سروس بنائی تھی۔ اسسٹنز کو اٹلی میں براہ راست پہنا کر اپنی کاروائی کا تحقیقی اور پروجیکティویng ٹالین میں شامل کرنا پڑن.
```

### Viwerڈنگ

یہ وائرال ٹرنر بنانے اور ان سروسز کو اسٹیل اور ماولنگ سیشن میں پرندوں کا پچھرانا متحرک ہے۔ 
```
**سروسز**
-------------

```
*   **لائڈ انجینائرنگ**: ذیل، ایک اسی جیسے ٹیکنالوجیز کی سروسز کو اسٹیل میں لائڈ کرنے کا تجربہ.
```

اس्पیس پلیٹ فارم پر:
*   **آئی این اے مہیا کرنے کی قوتی**: ایک ایسائین کا پروسیس جو کو انٹرنیٹ سروسز کا استعمال کرتے ہوئے ان اسکرپٹز ریکارڈ کرتے ہیں اور ان میں کو ایسٹھنڈائن ملازمت کا پروسیس شامل کرتا ہے.
*   **میگا میوزک**: ایک ماولنگ پلیٹ فارم جو لینکیوں، اسٹیل پلیٹ فارمز اور ریمس اینسٹریٹرز کی سروسز کو اپنی مہیا کرنے کے لیپڼے میں شامل کرتا ہے.

```
**بیکاروں کی خدمات**
-------------------------

```

اس سروس کو لائڈ کرنے والی مینجمنٹ، سٹریٹفکسنگ، برونگنگ، انسٹیلمیئر، اور فہم کا مشاورتی پروفیسر کے نگر میں شامل کھانے کے لیپڼے شامل کرکے ان سروسز کو اپنی مہیا کی گئی ہے۔
```

**ٹالینس**
----------------

```
*   **ریکیٹریسی**: ایک پلیٹ فارم جو انفراسٹرن میں شامل ہونے والی ٹیکنالوجیز کو اپنی مہیا کرتا ہے۔
*   **میکس ورکنگ اقدامات**: ایک ماولنگ پروسیسن جو بروننگ پلیٹ فارمز میں شامل ہو کر فیشنری انٹرینٹ ایجینٹس جیسے اناسومٹ کا استعمال کرتا ہے.
```

**اخration**
-------------

آپ اپنی فوجی اور خیر کار کی صلاحیت میں کام کرنے والی ٹیموں کو اس سروسز کا استعمال کریں جو مینجمنٹ نے وائرل ٹرنر بنائی تھی:
```
*   فوجی انٹرنیٹ ایجینٹس اپنی موarde پر پکڑ کر ایم میجر اور ملازمت کا پروسیسن بنانے کی سروسز کو استعمال کریں.
*   ماولنگ لیفٹلینز ساتھی کرنے کے لیپڼے بروئے کرائے جو فوج کے انسٹرکٹوڈ اور چیکبر کی سروسز کی موarde پر اس کا استعمال کرتے ہیں.


```
**بیلن میں اچھی طرح کام**
-----------------------------

آپ اپنی موarde کو بروئے کرائو جسکے بعد وائرل ٹرنر بنانے کی صلاحیت کا تجریبی پلیٹ فارم تلاش کریں:
```

لئی این اے مہیا کرنے کی قوتی کو لاحق کرکنے والے گارنیز:
```
*   **لئی این انجینائرنگ**: ایک مینس پلیٹ فارم جو ٹرینڈنگ کے اوponent 2009 میں لائڈ کیا گیا اور اسے لائڈ کرنے کے لیپڼے کی سروسز کو استعمال کرتے کوشش کرسکتی ہیں۔
```

اس ناویز میں، اسٹیل کی 2013 01 ایم اے مہیا کرنے کی قوتی:
```
*   **تینار 1.0 سروسز**: ایک سروسز کو لائڈ کرتا ہے جو ڈیبیو کروٹے گئے تھیس ناویز میں وائرل ٹرنر بنائی جاتی ہے اور تجربات کرنے، پلیٹ فارم پر ٹیکسچرس کا منبع استعمال کی جاتی ہے.

```
**دائمی انشورANS**
--------------------

اس اسٹیل میں 2 ملین مینجمنٹ کا ملازمت رکھتا ہے اور 50,000 ٹرینس کی انشورنس پیش کرنے کی سروسز فراہمیاتی میڈیا کو لانکیو نے 2016 میں بنایا، جو چیرلی پونل کی سرگرم کارروائی اور کٹنگ کی ایسی مشابہت کرتا ہے جو ساتھ ملاحظeten ہے۔
```

**ٹوازس کا شام**
-----------------

نائٹ اینگلیشٹر، سیزل گروپ اور انٹالٹی میں شیکوڈنگ:
```
*   لیبرٹری کراچ کا پلیٹ فارم
```

```
آکادمی کی صلاحیتوں
--------------------

ہوم اٹ لائٹ، ایبٹ ے میڈیا اور ایوب ریمس انٹرٹینمنٹ:
```

**ناویز**
---------

ویرال ٹرنر بنانے کے لیپڼوں کی فہرست:
```
*   الیکtranڈسک کا پروجنٹ
```

```
سٹیٹی ڈینٹل بوجھ 2019
```
```
دائیلی سلوک اور ذرات کے کوئران، ایبٹ ڑیو آف نائنگ
```
```
میمرگن، لیبرٹری پرو کی ٹرینس
```
```
پی ایچ جی کا پروجنٹ، ٹیکنالوجیز اور اسکرپٹز

```

### Cell 10: The Streaming Brochure (Final UX)

In [12]:
def stream_brochure(company_name, url):
    """
    Final function that fetches data and streams the brochure generation
    token-by-token for a better user experience.
    """
    # 1. First, do the research (This part is still synchronous)
    context_data = fetch_page_and_all_relevant_links(url)

    # 2. Start the streaming call to the LLM
    # We set stream=True to get chunks of tokens instead of a single response
    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url, context_data)}
          ],
        stream=True
    )

    # 3. Handle the incoming stream of tokens
    full_response = ""
    # Create a display handle to update the same Markdown block in real-time
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        # Extract the new token from the chunk delta
        token = chunk.choices[0].delta.content or ""
        full_response += token

        # Update the display in Colab as tokens arrive
        update_display(Markdown(full_response), display_id=display_handle.display_id)

# Execute the final streaming brochure
stream_brochure(company_name, target_url)

Selecting relevant links for https://huggingface.co/ by calling llama3.2:1b...
Found 3 relevant links.
Reading: about page at https://huggingface.co/...
Reading: company at https://huggingface.co/changelog...
Reading: facebook at https://huggingface.co/facebook...


### Hugging Face: The AI Community Building the Future

Welcome to Hugging Face, a platform where machine learning enthusiasts and professionals come together to collaborate on models, datasets, and applications. We are the home of the open-source StackDB, and our community is built on the principles of collaboration, innovation, and creativity.

**Unlimited Public Models, Datasets, and Applications**

At Hugging Face, we believe that everyone should have access to the power of machine learning. That's why we offer unlimited public models, datasets, and applications through our public API. Our platform allows developers to easily build, train, and deploy their own AI projects.

### Explore AI Apps

Browse 2M+ models and trending on this week's AI apps:

*   Lightricks/LTX-2: A model for text classification tasks, updated in 4 days ago.
*   fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA: Adjust camera angles in images using 3D controls or sliders, running on Zero, updated in 4 days ago.

### Models

Browse 2M+ models. You can find a model for any task or application, from text classification and sentence embedding to image generation and more:

*   tensel/HY-MT1.5-1.8B: A model for real-world data.
*   fal/Qwen-Image-Edit-2511-SLIM: Generate photorealistic images from text descriptions.

### Datasets

Access 21,076 datasets across various domains:

*   genrobot2025/10Kh-RealOmin-OpenData: Collect and share public datasets.

### Spaces

Deploy your model as a custom AI service on our platform. Spaces applications are open to collaboration and are highly scalable for large-scale production workloads:


Explore Models
-------------------------------------------------

Our platform provides unlimited access to over 2 million machine learning models, from text classification to generation. Browse the most up-to-date models across various domains:

*   Lightricks/LTX-2: A model for text classification tasks.
*   fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA: Adjust camera angles in images using 3D controls or sliders.

### Datasets
----------------------------------------------------------------

Browse 21,076 datasets from various fields:

*   genrobot2025/10Kh-RealOmin-OpenData: Collect and share public datasets.
*   HuggingFaceFW/finetranslations: A dataset collaboration platform for machine learning tasks.

### Spaces
---------------------------------------------------

Deploy your custom AI model as a service on our scalable platform, designed for large-scale production workloads:


### Careers & Jobs
------------------------

At Hugging Face, we are committed to creating inclusive community for everyone through:

*   Writing
*   Data Science
*   Cloud Computing

Join us and start building amazing projects with the open-source AI community framework.

### Get Started
-----------------

Click the button below to start your journey on Hugging Face. We look forward to seeing what you will make happen with our power of machine learning technology:
[Sign Up](https://huggingface.cosignup)
[Pricing & T&D](https://huggingface.co/pricing)